# Import packages

In [ ]:
pip install awkward

In [ ]:
pip install uproot

In [ ]:
pip install coffea

In [ ]:
import awkward as ak
import uproot
from coffea.nanoevents import NanoEventsFactory, NanoAODSchema, BaseSchema

# Read .ROOT files into a dask awkward array

In [ ]:
NanoAODSchema.warn_missing_crossrefs = False

fname = "https://raw.githubusercontent.com/CoffeaTeam/coffea/master/tests/samples/nano_dy.root"
events = NanoEventsFactory.from_root(
    {fname: "Events"},
    schemaclass=NanoAODSchema
).events()

In [ ]:
events.fields

In [ ]:
def getUpTree(uprobj, name):
    parts = name.split('/')
    # if len(parts) > 1:
    #     return getUpTree(uprobj, '/'.join(parts[1:]))
    return uprobj[name]

# Opening a root file without reading the contents

In [ ]:
tree_file = uproot.open(fname)

In [ ]:
ttree = getUpTree(tree_file, 'Events')

In [ ]:
events8881 = NanoEventsFactory.from_root(
    ttree,
    schemaclass=BaseSchema,
    entry_start = 0,
    entry_stop = 1,
    metadata={"dataset": "DYJets"}
).events()

## How can I rename luminosityBlock to Block? How can I basically map old fields to the new fields? This is because we can have fields TkEleL2_eta, GenEl_eta . I want to have the _eta part. This would be ideally done before the .compute() call.

In [ ]:
select = events8881[['luminosityBlock', 'event']]

select.compute()

# Trying to convert dask-awkward array > dask dataframe > call .rename() > dask array > dask-awkward array

In [ ]:
import dask.dataframe as dd
import dask_awkward as dak

In [ ]:
df = dak.to_dataframe(events)

name_map_new = {'run': 'gintas'}

df = df.rename(columns=name_map_new)

In [ ]:
df

In [ ]:
dask_array_simple = df.to_dask_array()

In [ ]:
type(dask_array_simple)

In [ ]:
dak_array = dak.from_dask_array(dask_array_simple)